In [40]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

nb_indicators=5

df=pd.read_csv("../data/dairy_cows.csv")
df.head()

,ID,Species,Animal_Class,Welfare_Hazards_Animal,Welfare_Hazards_Consequences,Welfare_Hazards_Impact,Ease_of_Hazard_Mitigation,Welfare_Indicator,Indicator_Ease,Indicator_Resources
0,1,Dairy cows,Tie stalls,Pasture access,Gastro-enteric disorders,High,Moderate,Abdominal discomfort,Moderate,Low
1,1,Dairy cows,Cubicles,Pasture access,Gastro-enteric disorders,High,Moderate,Abdominal discomfort,Moderate,Low
2,2,Dairy cows,Tie stalls,Continuous housing for long periods,General disruption of behaviour,Low,Difficult,Agonistic behaviour,Moderate,Low
3,2,Dairy cows,Cubicles,Continuous housing for long periods,General disruption of behaviour,Low,Difficult,Agonistic behaviour,Moderate,Low
4,3,Dairy cows,Tie stalls,Insufficient space,Restriction of movement,High,Moderate,Agonistic interactions,Moderate,Medium


In [41]:
ease_mapping = {'Easy': 1, 'Moderate': 2, 'Difficult': 3}
resources_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
impact_mapping = {'Low': 1, 'High': 2}

# Apply mappings
df['Indicator_Ease_Num'] = df['Indicator_Ease'].map(ease_mapping)
df['Ease_of_Hazard_Mitigation_Num'] = df['Ease_of_Hazard_Mitigation'].map(ease_mapping)
df['Indicator_Resources_Num'] = df['Indicator_Resources'].map(resources_mapping)
df['Welfare_Hazards_Impact_Num'] = df['Welfare_Hazards_Impact'].map(impact_mapping)

In [42]:
# List of unique Welfare Indicators
indicators_list = df['Welfare_Indicator'].unique().tolist()

# List of unique Welfare_Hazards_Animal
hazards_animal_list = df['Welfare_Hazards_Animal'].unique().tolist()

# List of unique Welfare_Hazards_Consequences
hazards_consequence_list = df['Welfare_Hazards_Consequences'].unique().tolist()

In [43]:
# Initialize a_hi_animal dictionary
a_hi_animal = {h: {i: 0 for i in indicators_list} for h in hazards_animal_list}

# Populate the dictionary
for idx, row in df.iterrows():
    h = row['Welfare_Hazards_Animal']
    i = row['Welfare_Indicator']
    a_hi_animal[h][i] = 1 

In [44]:
# Initialize a_ci_consequence dictionary
a_ci_consequence = {c: {i: 0 for i in indicators_list} for c in hazards_consequence_list}

# Populate the dictionary
for idx, row in df.iterrows():
    c = row['Welfare_Hazards_Consequences']
    i = row['Welfare_Indicator']
    a_ci_consequence[c][i] = 1  # Indicator i addresses consequence c

In [45]:
indicator_attributes = df.groupby('Welfare_Indicator').agg({
    'Indicator_Ease_Num': 'mean',
    'Ease_of_Hazard_Mitigation_Num': 'mean',
    'Indicator_Resources_Num': 'mean'
}).to_dict('index')

In [46]:
# Hazard impact for animal hazards
hazard_animal_impact = df.groupby('Welfare_Hazards_Animal')['Welfare_Hazards_Impact_Num'].mean().to_dict()

# Hazard impact for consequences
hazard_consequence_impact = df.groupby('Welfare_Hazards_Consequences')['Welfare_Hazards_Impact_Num'].mean().to_dict()

In [47]:
solver = pywraplp.Solver.CreateSolver('CBC')
solver.EnableOutput()


## prepear decision variables

In [48]:

# Decision variables for indicator selection
x_vars = {}
for i in indicators_list:
    x_vars[i] = solver.IntVar(0, 1, f'Select_Indicator_{i}')

# Decision variables for animal hazard coverage
y_vars_animal = {}
for h in hazards_animal_list:
    y_vars_animal[h] = solver.IntVar(0, 1, f'Cover_Hazard_Animal_{h}')

# Decision variables for consequence coverage
y_vars_consequence = {}
for c in hazards_consequence_list:
    y_vars_consequence[c] = solver.IntVar(0, 1, f'Cover_Hazard_Consequence_{c}')

In [49]:
solver.Add(solver.Sum([x_vars[i] for i in indicators_list]) == nb_indicators)


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11850ae80> >

In [50]:
#Hazard Coverage Constraints for Animal Hazards
for h in hazards_animal_list:
    # If any indicator covering hazard h is selected, y_vars_animal[h] must be 1
    indicators_covering_h = [x_vars[i] for i in indicators_list if a_hi_animal[h][i] == 1]
    solver.Add(y_vars_animal[h] <= solver.Sum(indicators_covering_h))
    solver.Add(y_vars_animal[h] * len(indicators_covering_h) >= solver.Sum(indicators_covering_h))

#Hazard Coverage Constraints for Consequences

for c in hazards_consequence_list:
    # If any indicator covering consequence c is selected, y_vars_consequence[c] must be 1
    indicators_covering_c = [x_vars[i] for i in indicators_list if a_ci_consequence[c][i] == 1]
    solver.Add(y_vars_consequence[c] <= solver.Sum(indicators_covering_c))
    solver.Add(y_vars_consequence[c] * len(indicators_covering_c) >= solver.Sum(indicators_covering_c))


## Objective Function
### Set Weights
To maximize the number of hazards and consequences explained, we'll prioritize maximizing their coverage in the objective function.

Set higher weights for hazard coverage terms.

In [51]:
weight_animal_hazard_coverage = 1 # High weight to prioritize hazard coverage
weight_consequence_coverage = 1    # High weight to prioritize consequence coverage

beta = 2    # Weight for Indicator_Ease
gamma = 1   # Weight for Ease_of_Hazard_Mitigation
delta = 1   # Weight for Indicator_Resources


In [52]:
# Objective function components
# Maximize the number of hazards and consequences covered
objective_terms = []

# Animal hazard coverage
for h in hazards_animal_list:
    objective_terms.append(weight_animal_hazard_coverage * y_vars_animal[h])

# Consequence coverage
for c in hazards_consequence_list:
    objective_terms.append(weight_consequence_coverage * y_vars_consequence[c])

# Penalize higher Indicator_Ease, Ease_of_Hazard_Mitigation, and Indicator_Resources
for i in indicators_list:
    indicator_penalty = (
        beta * indicator_attributes[i]['Indicator_Ease_Num'] +
        gamma * indicator_attributes[i]['Ease_of_Hazard_Mitigation_Num'] +
        delta * indicator_attributes[i]['Indicator_Resources_Num']
    )
    objective_terms.append(-indicator_penalty * x_vars[i])

# Set the objective function
solver.Maximize(solver.Sum(objective_terms))


In [53]:
status = solver.Solve()


Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Sep 13 2024 

command line - cbc -solve -quit (default strategy 1)
Presolve 91 (-46) rows, 85 (-38) columns and 502 (-188) elements
0  Obj -0 Primal inf 2.931988 (1) Dual inf 50866.684 (55)
14  Obj 49978.017
Optimal - objective value 49978.017
After Postsolve, objective 49978.017, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 49978.01667 - 14 iterations time 0.002, Presolve 0.00
Continuous objective value is 49978 - 0.00 seconds
Optimal - objective value 49978.017
Optimal - objective value 49978.017
Presolve 91 (-46) rows, 85 (-38) columns and 502 (-188) elements
0  Obj 49978.017 Dual inf 2870.3322 (6)
7  Obj 49978.017
Optimal - objective value 49978.017
Presolve 91 (0) rows, 85 (0) columns and 502 (0) elements
0  Obj 49978.017
Optimal - objective value 49978.017
0 fixed, 0 tightened bounds, 0 strengthened rows, 16 substitutions
Optimal - objective value 49978.017
Presolve 77 (-30) rows, 78 (-7) columns and 452

In [54]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
else:
    print('The solver did not find an optimal solution.')

Solution:


In [55]:
selected_indicators = [i for i in indicators_list if x_vars[i].solution_value() == 1]
print("\nSelected Welfare Indicators:")
for ind in selected_indicators:
    print(f"- {ind}")


Selected Welfare Indicators:
- Body condition scoring
- Calving records (death of cow)
- Distended udder
- Injuries
- Physiological stress indicators


In [56]:
covered_hazards_animal = [h for h in hazards_animal_list if y_vars_animal[h].solution_value() == 1]
print("\nCovered Animal Hazards:")
for h in covered_hazards_animal:
    print(f"- {h}")
print(f"Total Number of Animal Hazards Covered: {len(covered_hazards_animal)}")


Covered Animal Hazards:
- Continuous housing for long periods
- Inadequate flooring in passageways, feeding and milking areas
- Early separation of cow and calf
- Early separation of cow and calf (after 24h)
- Poor pasture quality
- Poor quality feed (low nutritive value)
- Poor quality feed (pathogens/toxins)
- Underfeeding
- Poor calving conditions (pen design)
- Delayed calving intervention
- Difficulty calving because of the sire
- Poor calving conditions (calving management)
- Being tied up temporarily
- Inadequate bedding (cubicle/stall floor)
- Inadequate design of waiting area (size, flooring, crowding gates)
- Inadequate maintenance of milking equipment
- Inadequate milking parlour design
- Poor cubicle design
- Poor stall design / stall too small
- Use of cow trainers
- Poor calving conditions (unable to separate from other animals)
- Lack of handling/restraining facilities
- Poor calving conditions (absence of pen)
- Milking robot breakdown
- Milking robot not used by cow
-

In [57]:
covered_hazards_consequence = [c for c in hazards_consequence_list if y_vars_consequence[c].solution_value() == 1]
print("\nCovered Consequences:")
for c in covered_hazards_consequence:
    print(f"- {c}")
print(f"Total Number of Consequences Covered: {len(covered_hazards_consequence)}")


Covered Consequences:
- Separation stress
- Prolonged hunger
- Mortality
- Environmental stress
- Group (social) stress
- Handling stress
- Soft tissue lesions and integument damage
- Cold stress
- Heat stress
- Bone lesions (incl. fractures and dislocations)
- Inability to chew and/or ruminate
- Isolation stress
Total Number of Consequences Covered: 12


In [58]:
print(f"\nObjective Function Value: {solver.Objective().Value()}")



Objective Function Value: 19.016666666666666


In [59]:
print("\nSelected Indicators and Their Attributes:")
for ind in selected_indicators:
    attrs = indicator_attributes[ind]
    print(f"- {ind}:")
    print(f"  Indicator_Ease_Num: {attrs['Indicator_Ease_Num']}")
    print(f"  Ease_of_Hazard_Mitigation_Num: {attrs['Ease_of_Hazard_Mitigation_Num']}")
    print(f"  Indicator_Resources_Num: {attrs['Indicator_Resources_Num']}")


Selected Indicators and Their Attributes:
- Body condition scoring:
  Indicator_Ease_Num: 2.0
  Ease_of_Hazard_Mitigation_Num: 1.5
  Indicator_Resources_Num: 1.0
- Calving records (death of cow):
  Indicator_Ease_Num: 1.0
  Ease_of_Hazard_Mitigation_Num: 1.3333333333333333
  Indicator_Resources_Num: 1.0
- Distended udder:
  Indicator_Ease_Num: 1.0
  Ease_of_Hazard_Mitigation_Num: 1.5
  Indicator_Resources_Num: 1.0
- Injuries:
  Indicator_Ease_Num: 1.0
  Ease_of_Hazard_Mitigation_Num: 2.5
  Indicator_Resources_Num: 1.0
- Physiological stress indicators:
  Indicator_Ease_Num: 3.0
  Ease_of_Hazard_Mitigation_Num: 2.15
  Indicator_Resources_Num: 3.0
